In [1]:
import numpy as np #linear algebra
import pandas as pd #data preprocessing
import matplotlib.pyplot as plt #data visualization
import h5py
import PIL

import utils.general_utils as util
from utils.general_utils import populate_breeds, get_imgMatrix_from_id, get_breed_value_from_id, get_filename_from_id
#other built utilities

from data_loader import dataFrameBuilder

#using inception_v3 to classify dog breeds
import tensorflow as tf #import tensroflow



In [2]:
#PARAMETERS
image_size = 500
batch_size = 100

In [3]:
%matplotlib inline

#prepare csv files
train = pd.read_csv("../data/included/labels.csv")
test = pd.read_csv("../data/included/test_id.csv")
BREED_LIST = "../data/preprocessed_data/breed_list.csv"

In [4]:
#prepare the breed list dataframe
labels = populate_breeds(BREED_LIST) #get the list of all dog breeds
labels_np = np.array(labels).reshape(120,1) #labels list reshaped to numpy array

In [5]:
#x = tf.placeholder(tf.float32, shape=[None, 500,500, 3], name='input_data')
x = tf.keras.layers.Input(shape=[500,500,3], batch_size=batch_size,name='input_data',dtype='float32')
y = tf.placeholder(tf.float32, shape=[None, 120], name='correct_labels')

y_pred = tf.placeholder(tf.float32, shape=[None,120], name='predicted_labels')

In [24]:
def train_input_fn(data_amnt = None):
    df = dataFrameBuilder(data_amnt)
    #df.shuffle().repeat().batch(batch_size)
    print(df.sample(1))
    return df.as_matrix(columns=['Image Data'])

def train_output_fn(data_amnt = None):
    df = dataFrameBuilder(data_amnt)
    return df.as_matrix(columns=['Breed'])


a = train_input_fn(100)

File:  1
                                 ID  \
0  0b3d3120a62c2bc280f589d473041d06   

                                          Image Data  \
0  [[[0, 0, 0], [0, 0, 0], [0, 0, 0], [0, 0, 0], ...   

                                               Breed  
0  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...  


In [7]:
#create an input layer
#input = tf.placeholder(tf.float32, [None, image_size*image_size])
#labels = tf.placeholder(tf.float32, [None, len(labels_np)])

#train_input_layer = tf.keras.layers.Input(tensor=x, name='input_layer')
#train_output_layer = tf.keras.layers.

In [8]:
#loss function
#loss = tf.keras
#optimizer_fn = tf.keras.optimizers.Adam()

#loss_fn = tf.keras.losses.categorical_hinge(y,
#                                           y_pred)

#loss_fn = tf.losses.hinge_loss(labels=y, logits=y_pred)

#loss_fn = tf.keras.losses.hinge(y_true=y, y_pred=y_pred)

In [9]:
inception_v3 = tf.keras.applications.InceptionV3(include_top=False,
                                                weights='imagenet',
                                                input_tensor=x,
                                                input_shape=[image_size,image_size,3],
                                                pooling='max',
                                                classes=120)

In [20]:
with tf.Session() as ss:
    #ss.run(inception_v3.compile(optimizer_fn,
     #                   loss_fn))
    img_data = train_input_fn(100)
    breed_data = train_output_fn(100)
    
    
    inception_v3.compile(loss=tf.keras.losses.mean_squared_error,
                        optimizer='sgd')
    inception_v3.build(_)
    #inception_v3.train_on_batch(x, y)
    #inception_v3.fit(x=img_data, y=breed_data, batch_size=batch_size)
    print(img_data[0])

File:  1
File:  1
[array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)]
